In [1]:
# We are going to be calculating explicitly the spectral curves for affine toda in the case of some specific
# choices of Lie algebra.
# We will then want to calculate the automorphism group of the spectral curve, hence we import the 
# RiemannSurface class. 
from sage.schemes.riemann_surfaces.riemann_surface import RiemannSurface

# Note that for all the calculations and formulae to make sense, one should read my Affine Toda notes.

In [8]:
# We start by specifying the rank (integer >=1) we want to deal with. 
rank = 4

# As a warning, Sage does not currently handle exceptional Lie algebras perfectly yet, so avoid them.
# Moreover, calculating the determinant becomes slow, so stick to algebras s.t. the fundamental representation
# has dimension <=10.

# Choose also the Cartan type (using the previously defined rank)
ct = ['A',rank]
g = LieAlgebra(QQ,cartan_type=ct,representation='matrix')

R = RootSystem(ct)
Theta = R.root_lattice().highest_root()

# What we care about in Theta is actually just its coefficients, so we will extract these into a list, along with 
# n0=1 to represent -Theta
ni = [1,] + [Theta[i] for i in (1..rank)]

# This assigns the matrices corresponding to the Chevalley basis in a list. We make E[0],F[0], correspond to the 
# highest root element, as if we are taking the extended affine Lie algebra. 
E = [matrix(g.highest_root_basis_elt(pos=False))]
F = [matrix(g.highest_root_basis_elt())]
H = []

N = E[0].ncols()
for i in (1..rank):
    E += [matrix(g.e(i))]
    F += [matrix(g.f(i))]
    H += [matrix(g.h(i))]
    
# a bit of initialisation of things that won't change
BR = QQ # base ring
# The variables r are motivated by the algebraic problem at hand, and can be simplified to the following case.
r = [1] + rank*[0,]

In [9]:
# We are now going to initialise the variables we are going to be using. 
# If Qsimplify, instead of the symbolic ring we work in a ring specifically constructed for the case in hand. 
Qsimplify = True

if Qsimplify:
    variables = ["p{}".format(i) for i in (1..rank)] + ["Q{}".format(i) for i in (0..rank)]
    R = PolynomialRing(BR,variables)
    R.inject_variables()
    Q = R.gens()[rank:2*rank+1]
    Qn = [Q[i]^ni[i] for i in (0..rank)]
    # The following ideal is motivated by the problem in hand, and simplifies the answer. 
    I = R.ideal(prod(Qn)-prod(ni))
    Rq = QuotientRing(R,I,variables)
    p = Rq.gens()[0:rank]
    Q = Rq.gens()[rank:2*rank+1]
    LR = LaurentPolynomialRing(Rq,['w','z'])
    LR.inject_variables()
else:
    p = [var("p{}".format(i), latex_name="p_{}".format(i)) for i in (1..rank)]
    w = var('w')
    z = var('z')
    Q = [var("Q{}".format(i), latex_name="Q_{}".format(i)) for i in (0..rank)]
    Qn = [Q[i]^ni[i] for i in (0..rank)]

# r are the exponents defined above. 
X = [z^r[i]*E[i] + z^(-r[i])*F[i] for i in (0..rank)]
L = [Q[0]*X[0],] + [-p[i-1]*H[i-1] + Q[i]*X[i] for i in (1..rank)]
L = sum(L)
# L is now the Lax matrix of the affine Toda system, up to some simplifying assumptions.

# C is effectively the characteristic polynomial, calculated with det. 
nc = L.ncols() 
C = L - w*matrix.identity(nc)
C = C.det()

# This imposes the condition that prod(Qn) = prod(ni), done automatically in the quotient ring.
# This is motivated by the problem that is being considered in general and makes the answer clearer. 
if Qsimplify:
    D = C
else:
    D = C.subs({Q0:prod(ni)/prod(Qn[1:])}).simplify_full()

# This extracts the information about the coefficients of the polynomial, as next we will reconstruct
# the polynomial but susbstituting for coefficients that are unwieldy.
if not Qsimplify:    
    coeffz = D.collect(z).coefficients(z)
    exponents = []
    coeffs = []
    for k in range(len(coeffz)):
        coeffzkw = coeffz[k][0].collect(w).coefficients(w)
        for j in range(len(coeffzkw)):
            exponents += [[coeffzkw[j][1],coeffz[k][1]]]
            coeffs += [coeffzkw[j][0]]
else:
    coeffs = D.coefficients()
    exponents = D.exponents()

# We automate some substitution to make the resulting curve clearer.
# This script is written with the knowledge that we expect the exponents of z to be -1,0,1, 
nconsts = len(exponents)
c = [var("c{}".format(i), latex_name="c_{}".format(i)) for i in (1..nconsts)]
Dred = 0
A.<x,y> = BR[]
j = 0
yemin = min([e[0] for e in exponents])
xemin = min([e[1] for e in exponents])
for i in range(nconsts):
    cj = coeffs[i]
    if (cj.variables()==()):
        Dred += A(cj)*y^(exponents[i][0]-yemin)*x^(exponents[i][1]-xemin)
    else:
        Dred += c[j]*y^(exponents[i][0]-yemin)*x^(exponents[i][1]-xemin)
        j += 1
total_consts = j
# We now see a form of the spectral curve, where the coefficients have been replaced with some variables c
display(Dred.collect(y))
f = Dred

# In order to calculate the automorphism group, we cannot have symbolic coefficients, so we make an arbitrary 
# choice. To get evidence that you haven't found an exceptional form of the spectral curve with additional symmetry
# multiple substitutions should be tried. 
for j in range(total_consts):
    f = (f.subs({c[j]:2*j})).simplify_full()
S = RiemannSurface(A(f))
G = S.symplectic_automorphism_group()
print(G.structure_description())

Defining p1, p2, p3, p4, Q0, Q1, Q2, Q3, Q4
Defining w, z


-x*y^5 + c1*x*y^3 + c2*x*y^2 + c3*x*y + c4*x - x^2 + 1

C2
